In this notebook, we use the machine learning model to generate simulations (i.e. produce new synthetic datasets) that can be used to extract insights. This type of simulation study can be potentially useful for substituting bench experiments, which can be prohibitive in terms of time and cost.

General idea:
- train the ML model on Limonene data
- use the trained ML model to simulate virtual strains
- analyze the simulated data to answer relevant questions that can improve biofuel production (e.g. which proteins to over/underexpress?)

In [1]:
import sys
sys.path.append("../tool")

from learn_biological import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

In [2]:
# Define states and controls
proteins = ['AtoB', 'HMGS', 'HMGR', 'MK', 'PMK', 'PMD', 'Idi','GPPS', 'LS']
metabolites = ['GPPS', 'LS', 'Acetyl-CoA', 'Acetoacetyl-CoA', 'HMG-CoA', 'Mev', 'MevP','MevPP', 'IPP', 'DMAPP', 'GPP', 'Limonene']

In [3]:
# Load data 
df = read_timeseries_data('../data/limonene_simulated_n10000.csv',
                          metabolites,proteins,augment=None,n=100)

In [4]:
# Sample 100 strains 
strains = df.index.get_level_values(0).unique()

# Define train and test set
test_df = df.loc[df.index.get_level_values(0).isin(strains[0:50])]
train_df = df.loc[df.index.get_level_values(0).isin(strains[50:])]

In [5]:
# Define dynamical model
model = learn_dynamics(train_df,generations=10,population_size=10,verbose=True)

Training progress...
Training progress...
Training progress...
Training progress...
Training progress...
Training progress...
Training progress...
Training progress...
Training progress...
Training progress...
Training progress...
Training progress...


In [6]:
# Use model to get the trajectory 
trajectory_df = test_df.groupby('Strain').apply(lambda x: simulate_dynamics(model,x,tolerance=5e-4))
display(trajectory_df)

Display strain...


states                                                     \
                     GPPS            LS Acetyl-CoA Acetoacetyl-CoA   HMG-CoA   
Strain Time                                                                    
8      0     5.243580e+00  4.567956e+00   0.199750        0.200886  0.199663   
       1     3.367088e+06  2.254286e+06   0.199880        0.198524  0.199239   
       2     4.363593e+06  2.882618e+06   0.205488        0.200694  0.197295   
       3     4.880791e+06  3.156385e+06   0.205923        0.200311  0.197031   
       4     5.126960e+06  3.325890e+06   0.209332        0.198882  0.193720   
...                   ...           ...        ...             ...       ...   
       67    6.212313e+06  3.853893e+06   0.333682        0.207145  0.116175   
       68    6.215943e+06  3.909248e+06   0.329281        0.207573  0.116803   
       69    6.236064e+06  3.885030e+06   0.339425        0.207360  0.115346   
       70    6.270190e+06  3.931585e+06   0.335347        0.207144  0.113665   
       71    6.286109e+06  3.851905e+06   0.343338        0.205832  0.112915   

                                                               ...  \
                  Mev      MevP     MevPP       IPP     DMAPP  ...   
Strain Time                                                    ...   
8      0     0.199567  0.200015  0.200910  0.198619  0.201979  ...   
       1     0.198873  0.198596  0.188449  0.212188  0.201056  ...   
       2     0.197751  0.203111  0.174766  0.224885  0.201309  ...   
       3     0.202089  0.200594  0.161355  0.238160  0.199548  ...   
       4     0.201118  0.202491  0.148521  0.254343  0.201254  ...   
...               ...       ...       ...       ...       ...  ...   
       67    0.205641  0.239794  0.006375  0.433120  0.200754  ...   
       68    0.205709  0.238575  0.006336  0.434997  0.198305  ...   
       69    0.205690  0.238096  0.006302  0.434515  0.198525  ...   
       70    0.206087  0.236499  0.006307  0.434416  0.198995  ...   
       71    0.203642  0.240812  0.006320  0.430410  0.201758  ...   

            derivatives                                                \
             Acetyl-CoA Acetoacetyl-CoA   HMG-CoA       Mev      MevP   
Strain Time                                                             
8      0       0.002283       -0.000342 -0.000938 -0.000307  0.001054   
       1       0.002303       -0.000159 -0.001164  0.000097  0.000814   
       2       0.002322        0.000024 -0.001390  0.000501  0.000574   
       3       0.002342        0.000208 -0.001616  0.000905  0.000335   
       4       0.002276        0.000342 -0.001347  0.000733  0.000422   
...                 ...             ...       ...       ...       ...   
       67      0.001870        0.000127 -0.000949 -0.000080  0.000178   
       68      0.002374       -0.000031 -0.000936 -0.000310  0.000322   
       69      0.002680       -0.000392 -0.001052 -0.000527  0.000172   
       70      0.002987       -0.000754 -0.001169 -0.000743  0.000023   
       71      0.003294       -0.001116 -0.001285 -0.000960 -0.000127   

                                                               
                MevPP       IPP     DMAPP       GPP  Limonene  
Strain Time                                                    
8      0    -0.014058  0.013905 -0.000731  0.001424  0.000205  
       1    -0.013655  0.013558 -0.000540  0.000868  0.000239  
       2    -0.013253  0.013212 -0.000350  0.000313  0.000274  
       3    -0.012850  0.012866 -0.000159 -0.000243  0.000309  
       4    -0.012300  0.012468 -0.000009 -0.000447  0.000339  
...               ...       ...       ...       ...       ...  
       67   -0.000006  0.000646 -0.000362 -0.000383  0.000383  
       68   -0.000005 -0.000963 -0.000153 -0.000349  0.000389  
       69   -0.000004 -0.000718  0.000467 -0.000368  0.000377  
       70   -0.000004 -0.000472  0.001086 -0.000387  0.000364  
       71   -0.000004 -0.000226  0.001705 -0.000406  0.000352 

Integrating ODE...
Display trajectory...


,GPPS,LS,Acetyl-CoA,Acetoacetyl-CoA,HMG-CoA,Mev,MevP,MevPP,IPP,DMAPP,GPP,Limonene,Time
0,5.243580e+00,4.567956e+00,0.199750,0.200886,0.199663,0.199567,0.200015,0.200910,0.198619,0.201979,0.197974,0.000000,0
1,1.084793e+06,9.571640e+05,0.201800,0.200990,0.199092,0.199732,0.199990,0.193096,0.206514,0.201702,0.198207,0.000196,1
2,2.174874e+06,1.611340e+06,0.203808,0.201126,0.197923,0.200063,0.200332,0.180482,0.221248,0.201629,0.198179,0.000335,2
3,3.059126e+06,2.202966e+06,0.205807,0.201261,0.196753,0.200421,0.200715,0.167696,0.235902,0.201591,0.197890,0.000519,3
4,3.560746e+06,2.526719e+06,0.207804,0.201387,0.195584,0.200802,0.201123,0.155487,0.248981,0.201570,0.197620,0.000750,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,4.383001e+06,2.991583e+06,0.331404,0.210803,0.122395,0.206795,0.229165,0.007858,0.459237,0.201399,0.177202,0.018109,67
68,4.381401e+06,2.992087e+06,0.333336,0.210954,0.121245,0.206589,0.229601,0.007644,0.460179,0.201446,0.176939,0.018387,68
69,4.379166e+06,2.992563e+06,0.335266,0.211109,0.120095,0.206361,0.230038,0.007430,0.461137,0.201501,0.176677,0.018665,69
70,4.376809e+06,2.993093e+06,0.337194,0.211270,0.118944,0.206127,0.230472,0.007216,0.462095,0.201587,0.176414,0.018944,70


Display strain...


...

Display strain...


states                                                     \
                     GPPS            LS Acetyl-CoA Acetoacetyl-CoA   HMG-CoA   
Strain Time                                                                    
4372   0     4.900874e+00  8.566427e+00   0.200240        0.201982  0.201385   
       1     2.750941e+06  6.957545e+06   0.200593        0.200097  0.200083   
       2     3.501255e+06  7.549072e+06   0.206608        0.198502  0.199531   
       3     3.935013e+06  7.616499e+06   0.204087        0.200529  0.197675   
       4     4.142850e+06  7.894550e+06   0.209062        0.199940  0.196815   
...                   ...           ...        ...             ...       ...   
       67    4.916555e+06  8.252424e+06   0.328568        0.213539  0.138359   
       68    4.950851e+06  8.135281e+06   0.333569        0.213966  0.137907   
       69    4.859736e+06  8.211690e+06   0.333725        0.213675  0.137946   
       70    4.923822e+06  8.230192e+06   0.337292        0.217168  0.135779   
       71    4.952157e+06  8.166857e+06   0.337823        0.217831  0.135936   

                                                               ...  \
                  Mev      MevP     MevPP       IPP     DMAPP  ...   
Strain Time                                                    ...   
4372   0     0.200503  0.201497  0.200252  0.200564  0.200502  ...   
       1     0.200205  0.199746  0.187252  0.214248  0.200193  ...   
       2     0.198187  0.203045  0.166558  0.234367  0.198651  ...   
       3     0.201713  0.197502  0.147457  0.256715  0.200026  ...   
       4     0.198640  0.197539  0.132099  0.275008  0.198336  ...   
...               ...       ...       ...       ...       ...  ...   
       67    0.207654  0.139334  0.007444  0.505903  0.200436  ...   
       68    0.208094  0.136192  0.007283  0.515175  0.198442  ...   
       69    0.205566  0.134758  0.007301  0.514348  0.202254  ...   
       70    0.207173  0.135233  0.007214  0.511731  0.199704  ...   
       71    0.206062  0.136615  0.007269  0.516633  0.198876  ...   

            derivatives                                                \
             Acetyl-CoA Acetoacetyl-CoA   HMG-CoA       Mev      MevP   
Strain Time                                                             
4372   0       0.001852       -0.001753 -0.001539 -0.000749 -0.000140   
       1       0.001965       -0.001177 -0.001263 -0.000430 -0.000513   
       2       0.002077       -0.000601 -0.000987 -0.000112 -0.000886   
       3       0.002190       -0.000025 -0.000710  0.000207 -0.001259   
       4       0.002301        0.000216 -0.000833  0.000669 -0.001348   
...                 ...             ...       ...       ...       ...   
       67      0.001585        0.000825 -0.000908 -0.000268 -0.001058   
       68      0.001937        0.001080 -0.000908 -0.000254 -0.000821   
       69      0.002125        0.001359 -0.000793 -0.000451 -0.000464   
       70      0.002313        0.001638 -0.000677 -0.000649 -0.000108   
       71      0.002501        0.001917 -0.000561 -0.000846  0.000249   

                                                               
                MevPP       IPP     DMAPP       GPP  Limonene  
Strain Time                                                    
4372   0    -0.018598  0.020159 -0.001204 -0.001426  0.000671  
       1    -0.018092  0.019643 -0.000789 -0.001180  0.000703  
       2    -0.017587  0.019127 -0.000374 -0.000934  0.000736  
       3    -0.017081  0.018611  0.000041 -0.000689  0.000768  
       4    -0.016431  0.018323 -0.000130 -0.000655  0.000803  
...               ...       ...       ...       ...       ...  
       67   -0.000057  0.002488  0.000057 -0.000995  0.000657  
       68   -0.000048  0.001753 -0.000009 -0.000706  0.000596  
       69   -0.000038  0.001264 -0.000349 -0.000655  0.000566  
       70   -0.000028  0.000775 -0.000690 -0.000604  0.000536  
       71   -0.000018  0.000287 -0.001031 -0.000553  0.000506 

Integrating ODE...
Display trajectory...


,GPPS,LS,Acetyl-CoA,Acetoacetyl-CoA,HMG-CoA,Mev,MevP,MevPP,IPP,DMAPP,GPP,Limonene,Time
0,4.900874e+00,8.566427e+00,0.200240,0.201982,0.201385,0.200503,0.201497,0.200252,0.200564,0.200502,0.200700,0.000000,0
1,9.256536e+05,8.092486e+05,0.202276,0.202029,0.200898,0.200512,0.201061,0.190505,0.210776,0.200392,0.200505,0.000123,1
2,1.818213e+06,1.292621e+06,0.204258,0.202187,0.200238,0.200564,0.200302,0.175017,0.227532,0.200361,0.200197,0.000146,2
3,2.605394e+06,1.748294e+06,0.206223,0.202363,0.199423,0.200692,0.199417,0.159493,0.244294,0.200333,0.199838,0.000216,3
4,3.110924e+06,2.058850e+06,0.208179,0.202536,0.198571,0.200865,0.198456,0.144019,0.261056,0.200318,0.199442,0.000329,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,4.562151e+06,2.911280e+06,0.327641,0.214884,0.141770,0.208884,0.138501,0.000895,0.477882,0.200114,0.171527,0.015416,67
68,4.568316e+06,2.914938e+06,0.329495,0.215080,0.140952,0.208862,0.137738,0.000681,0.478123,0.200128,0.171068,0.015699,68
69,4.573978e+06,2.917002e+06,0.331348,0.215271,0.140168,0.208829,0.136973,0.000467,0.478326,0.200132,0.170625,0.015982,69
70,4.578864e+06,2.918994e+06,0.333199,0.215460,0.139383,0.208793,0.136211,0.000253,0.478528,0.200153,0.170187,0.016264,70


GPPS            LS  Acetyl-CoA  Acetoacetyl-CoA   HMG-CoA  \
Strain                                                                         
8      0   5.243580e+00  4.567956e+00    0.199750         0.200886  0.199663   
       1   1.084793e+06  9.571640e+05    0.201800         0.200990  0.199092   
       2   2.174874e+06  1.611340e+06    0.203808         0.201126  0.197923   
       3   3.059126e+06  2.202966e+06    0.205807         0.201261  0.196753   
       4   3.560746e+06  2.526719e+06    0.207804         0.201387  0.195584   
...                 ...           ...         ...              ...       ...   
4372   67  4.562151e+06  2.911280e+06    0.327641         0.214884  0.141770   
       68  4.568316e+06  2.914938e+06    0.329495         0.215080  0.140952   
       69  4.573978e+06  2.917002e+06    0.331348         0.215271  0.140168   
       70  4.578864e+06  2.918994e+06    0.333199         0.215460  0.139383   
       71  4.583194e+06  2.920994e+06    0.335050         0.215653  0.138598   

                Mev      MevP     MevPP       IPP     DMAPP       GPP  \
Strain                                                                  
8      0   0.199567  0.200015  0.200910  0.198619  0.201979  0.197974   
       1   0.199732  0.199990  0.193096  0.206514  0.201702  0.198207   
       2   0.200063  0.200332  0.180482  0.221248  0.201629  0.198179   
       3   0.200421  0.200715  0.167696  0.235902  0.201591  0.197890   
       4   0.200802  0.201123  0.155487  0.248981  0.201570  0.197620   
...             ...       ...       ...       ...       ...       ...   
4372   67  0.208884  0.138501  0.000895  0.477882  0.200114  0.171527   
       68  0.208862  0.137738  0.000681  0.478123  0.200128  0.171068   
       69  0.208829  0.136973  0.000467  0.478326  0.200132  0.170625   
       70  0.208793  0.136211  0.000253  0.478528  0.200153  0.170187   
       71  0.208748  0.135469  0.000040  0.478739  0.200182  0.169750   

           Limonene  Time  
Strain                     
8      0   0.000000     0  
       1   0.000196     1  
       2   0.000335     2  
       3   0.000519     3  
       4   0.000750     4  
...             ...   ...  
4372   67  0.015416    67  
       68  0.015699    68  
       69  0.015982    69  
       70  0.016264    70  
       71  0.016548    71  

[3600 rows x 13 columns]

In [8]:
X = test_df['controls'].loc[trajectory_df.index.get_level_values(1)==69]
y = test_df['states']['Limonene'].loc[trajectory_df.index.get_level_values(1)==69]
y_p = trajectory_df['Limonene'].loc[trajectory_df.index.get_level_values(1)==69]

In [15]:
# Set up the df of features,targets and predicted columns
pd.concat([X,y,y_p],axis=1).to_csv('../data/predicted_data.csv')

In [16]:
trajectory_df.to_csv('../data/full_timeseries_predicted.csv')

,Strain,Time,AtoB,HMGS,HMGR,MK,PMK,PMD,Idi,GPPS,LS,Limonene,Limonene.1
34,3377,69,5.926473e+06,7.989306e+06,5.894520e+06,3.729601e+06,4.761092e+06,4.386285e+06,1.475749e+06,7.362698e+06,7.392274e+06,0.046160,0.010606
46,4254,69,2.641246e+06,4.717867e+06,6.024744e+06,5.331594e+06,9.805328e+06,4.632462e+06,4.984441e+06,8.695452e+06,4.147104e+06,0.026884,0.011293
21,2275,69,6.442751e+06,9.002996e+06,6.631622e+06,1.642760e+06,7.789306e+06,7.719690e+06,5.714156e+06,4.113871e+06,4.292400e+06,0.028059,0.012405
24,2470,69,3.993804e+06,9.233155e+06,6.086413e+06,5.341812e+06,9.076154e+06,3.614046e+06,2.604519e+06,1.479142e+06,9.341756e+06,0.056022,0.012731
4,421,69,1.457053e+06,7.816741e+06,8.234266e+06,1.803335e+06,9.469114e+06,2.652185e+06,9.645477e+06,8.771075e+06,2.417513e+06,0.016111,0.013122
39,3740,69,2.675863e+06,6.200234e+06,7.237677e+06,7.480112e+06,7.117685e+06,5.346569e+06,5.110414e+06,7.637373e+06,8.642417e+06,0.053085,0.013976
38,3723,69,1.795929e+06,8.089002e+06,4.100562e+06,6.294429e+06,7.698795e+06,2.918365e+06,5.372484e+06,6.607267e+06,3.378702e+06,0.022221,0.014115
37,3703,69,7.408448e+06,3.960825e+06,1.734006e+06,1.535192e+06,8.267712e+06,7.510189e+06,8.853504e+06,1.357129e+06,6.558708e+06,0.040399,0.014203
33,3174,69,8.741548e+06,5.801273e+06,9.471622e+06,4.936268e+06,7.861117e+06,8.785788e+06,3.657470e+06,5.432179e+06,6.984913e+06,0.043241,0.014424
14,1092,69,5.461887e+06,1.391992e+06,6.586154e+06,3.874067e+06,6.808475e+06,3.738435e+06,4.997220e+06,6.680690e+06,5.853272e+06,0.035742,0.015626
